# 🚑 AmbuRoute - Testing & Validation
## Phase 5: Comprehensive Testing and Performance Validation

This notebook provides comprehensive testing and validation for the AmbuRoute system, including accuracy testing, performance benchmarking, and system validation.


In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import json
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import warnings
warnings.filterwarnings('ignore')

# Import YOLOv5 and other ML libraries
from ultralytics import YOLO
import torch

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🧪 Testing & Validation Environment Ready!")
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")


🧪 Testing & Validation Environment Ready!
🔧 PyTorch version: 2.8.0+cpu
🎮 CUDA available: False


## 🧪 Comprehensive Testing Framework


In [2]:
class AmbuRouteTester:
    """Comprehensive testing framework for AmbuRoute system"""
    
    def __init__(self, model_path, test_data_path="dataset/images/test"):
        self.model_path = model_path
        self.test_data_path = Path(test_data_path)
        self.model = YOLO(model_path)
        
        # Test results storage
        self.test_results = {
            'detection_metrics': {},
            'performance_metrics': {},
            'accuracy_metrics': {},
            'system_metrics': {}
        }
        
        print(f"✅ AmbuRoute Tester initialized with model: {model_path}")
    
    def test_detection_accuracy(self, confidence_threshold=0.5, iou_threshold=0.6):
        """Test detection accuracy on test dataset"""
        print("🔍 Testing detection accuracy...")
        
        # Get test images
        test_images = list(self.test_data_path.glob("*.jpg")) + list(self.test_data_path.glob("*.png"))
        
        if not test_images:
            print("❌ No test images found!")
            return None
        
        print(f"📊 Testing on {len(test_images)} images")
        
        # Initialize metrics
        true_positives = 0
        false_positives = 0
        false_negatives = 0
        true_negatives = 0
        
        detection_results = []
        
        for img_path in test_images:
            # Load image
            image = cv2.imread(str(img_path))
            if image is None:
                continue
            
            # Run detection
            results = self.model(image, conf=confidence_threshold, iou=iou_threshold)
            
            # Check for ground truth (simplified - assumes filename contains 'ambulance' if positive)
            has_ambulance_gt = 'ambulance' in img_path.stem.lower()
            
            # Check for detections
            detected_ambulance = False
            detections = []
            
            for result in results:
                if result.boxes is not None:
                    for box in result.boxes:
                        class_id = int(box.cls[0])
                        confidence = float(box.conf[0])
                        
                        if class_id == 0:  # Ambulance class
                            detected_ambulance = True
                            detections.append({
                                'confidence': confidence,
                                'bbox': box.xyxy[0].cpu().numpy()
                            })
            
            # Calculate metrics
            if has_ambulance_gt and detected_ambulance:
                true_positives += 1
            elif not has_ambulance_gt and detected_ambulance:
                false_positives += 1
            elif has_ambulance_gt and not detected_ambulance:
                false_negatives += 1
            else:
                true_negatives += 1
            
            # Store results
            detection_results.append({
                'image_path': str(img_path),
                'ground_truth': has_ambulance_gt,
                'detected': detected_ambulance,
                'detections': detections
            })
        
        # Calculate metrics
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)
        
        # Store results
        self.test_results['detection_metrics'] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'accuracy': accuracy,
            'true_positives': true_positives,
            'false_positives': false_positives,
            'false_negatives': false_negatives,
            'true_negatives': true_negatives,
            'total_images': len(test_images)
        }
        
        print(f"✅ Detection accuracy testing completed!")
        print(f"   Precision: {precision:.3f}")
        print(f"   Recall: {recall:.3f}")
        print(f"   F1-Score: {f1:.3f}")
        print(f"   Accuracy: {accuracy:.3f}")
        
        return self.test_results['detection_metrics']
    
    def benchmark_performance(self, num_iterations=100, image_size=(640, 640)):
        """Benchmark model performance"""
        print("⚡ Benchmarking model performance...")
        
        # Create dummy image
        dummy_image = np.random.randint(0, 255, (*image_size, 3), dtype=np.uint8)
        
        # Warm up
        print("🔥 Warming up model...")
        for _ in range(10):
            _ = self.model(dummy_image)
        
        # Benchmark inference time
        print(f"📊 Running {num_iterations} inference iterations...")
        inference_times = []
        
        for i in range(num_iterations):
            start_time = time.time()
            _ = self.model(dummy_image)
            end_time = time.time()
            inference_times.append(end_time - start_time)
        
        # Calculate metrics
        avg_inference_time = np.mean(inference_times)
        std_inference_time = np.std(inference_times)
        min_inference_time = np.min(inference_times)
        max_inference_time = np.max(inference_times)
        fps = 1 / avg_inference_time
        
        # Memory usage (if available)
        memory_usage = 0
        if torch.cuda.is_available():
            memory_usage = torch.cuda.memory_allocated() / 1024**3  # GB
        
        # Store results
        self.test_results['performance_metrics'] = {
            'avg_inference_time': avg_inference_time,
            'std_inference_time': std_inference_time,
            'min_inference_time': min_inference_time,
            'max_inference_time': max_inference_time,
            'fps': fps,
            'memory_usage_gb': memory_usage,
            'iterations': num_iterations
        }
        
        print(f"✅ Performance benchmarking completed!")
        print(f"   Average inference time: {avg_inference_time*1000:.2f} ms")
        print(f"   FPS: {fps:.2f}")
        print(f"   Memory usage: {memory_usage:.2f} GB")
        
        return self.test_results['performance_metrics']
    
    def test_confidence_thresholds(self, thresholds=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
        """Test model performance across different confidence thresholds"""
        print("🎯 Testing confidence thresholds...")
        
        threshold_results = []
        
        for threshold in thresholds:
            print(f"   Testing threshold: {threshold}")
            
            # Get test images
            test_images = list(self.test_data_path.glob("*.jpg")) + list(self.test_data_path.glob("*.png"))
            
            if not test_images:
                continue
            
            # Test with current threshold
            true_positives = 0
            false_positives = 0
            false_negatives = 0
            
            for img_path in test_images[:50]:  # Limit for speed
                # Load image
                image = cv2.imread(str(img_path))
                if image is None:
                    continue
                
                # Run detection
                results = self.model(image, conf=threshold)
                
                # Check ground truth
                has_ambulance_gt = 'ambulance' in img_path.stem.lower()
                
                # Check detections
                detected_ambulance = False
                for result in results:
                    if result.boxes is not None:
                        for box in result.boxes:
                            class_id = int(box.cls[0])
                            if class_id == 0:  # Ambulance class
                                detected_ambulance = True
                                break
                
                # Update metrics
                if has_ambulance_gt and detected_ambulance:
                    true_positives += 1
                elif not has_ambulance_gt and detected_ambulance:
                    false_positives += 1
                elif has_ambulance_gt and not detected_ambulance:
                    false_negatives += 1
            
            # Calculate metrics
            precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
            recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            threshold_results.append({
                'threshold': threshold,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'true_positives': true_positives,
                'false_positives': false_positives,
                'false_negatives': false_negatives
            })
        
        # Store results
        self.test_results['threshold_analysis'] = threshold_results
        
        print(f"✅ Confidence threshold testing completed!")
        
        return threshold_results
    
    def generate_test_report(self):
        """Generate comprehensive test report"""
        print("📋 Generating test report...")
        
        report = {
            'timestamp': datetime.now().isoformat(),
            'model_path': str(self.model_path),
            'test_results': self.test_results
        }
        
        # Save report
        report_path = Path("results") / "test_report.json"
        report_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(report_path, 'w') as f:
            json.dump(report, f, indent=2)
        
        print(f"✅ Test report saved to: {report_path}")
        
        return report
    
    def plot_test_results(self):
        """Plot comprehensive test results"""
        print("📊 Plotting test results...")
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('AmbuRoute Test Results', fontsize=16, fontweight='bold')
        
        # Plot 1: Detection Metrics
        if 'detection_metrics' in self.test_results:
            metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy']
            values = [
                self.test_results['detection_metrics']['precision'],
                self.test_results['detection_metrics']['recall'],
                self.test_results['detection_metrics']['f1_score'],
                self.test_results['detection_metrics']['accuracy']
            ]
            
            bars = axes[0,0].bar(metrics, values, color=['blue', 'red', 'green', 'orange'])
            axes[0,0].set_title('Detection Metrics')
            axes[0,0].set_ylabel('Score')
            axes[0,0].set_ylim(0, 1)
            
            # Add value labels
            for bar, value in zip(bars, values):
                height = bar.get_height()
                axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                              f'{value:.3f}', ha='center', va='bottom')
        
        # Plot 2: Performance Metrics
        if 'performance_metrics' in self.test_results:
            perf_metrics = ['Avg Inference (ms)', 'FPS', 'Memory (GB)']
            perf_values = [
                self.test_results['performance_metrics']['avg_inference_time'] * 1000,
                self.test_results['performance_metrics']['fps'],
                self.test_results['performance_metrics']['memory_usage_gb']
            ]
            
            bars = axes[0,1].bar(perf_metrics, perf_values, color=['purple', 'cyan', 'yellow'])
            axes[0,1].set_title('Performance Metrics')
            axes[0,1].set_ylabel('Value')
            
            # Add value labels
            for bar, value in zip(bars, perf_values):
                height = bar.get_height()
                axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                              f'{value:.2f}', ha='center', va='bottom')
        
        # Plot 3: Confusion Matrix
        if 'detection_metrics' in self.test_results:
            cm_data = [
                [self.test_results['detection_metrics']['true_positives'], 
                 self.test_results['detection_metrics']['false_positives']],
                [self.test_results['detection_metrics']['false_negatives'], 
                 self.test_results['detection_metrics']['true_negatives']]
            ]
            
            sns.heatmap(cm_data, annot=True, fmt='d', ax=axes[0,2], 
                       xticklabels=['Predicted Positive', 'Predicted Negative'],
                       yticklabels=['Actual Positive', 'Actual Negative'])
            axes[0,2].set_title('Confusion Matrix')
        
        # Plot 4: Confidence Threshold Analysis
        if 'threshold_analysis' in self.test_results:
            thresholds = [r['threshold'] for r in self.test_results['threshold_analysis']]
            precision_vals = [r['precision'] for r in self.test_results['threshold_analysis']]
            recall_vals = [r['recall'] for r in self.test_results['threshold_analysis']]
            f1_vals = [r['f1_score'] for r in self.test_results['threshold_analysis']]
            
            axes[1,0].plot(thresholds, precision_vals, 'o-', label='Precision', color='blue')
            axes[1,0].plot(thresholds, recall_vals, 'o-', label='Recall', color='red')
            axes[1,0].plot(thresholds, f1_vals, 'o-', label='F1-Score', color='green')
            axes[1,0].set_title('Confidence Threshold Analysis')
            axes[1,0].set_xlabel('Confidence Threshold')
            axes[1,0].set_ylabel('Score')
            axes[1,0].legend()
            axes[1,0].grid(True)
        
        # Plot 5: Performance Distribution
        if 'performance_metrics' in self.test_results:
            # Simulate inference time distribution
            mean_time = self.test_results['performance_metrics']['avg_inference_time']
            std_time = self.test_results['performance_metrics']['std_inference_time']
            times = np.random.normal(mean_time, std_time, 1000)
            
            axes[1,1].hist(times * 1000, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
            axes[1,1].axvline(mean_time * 1000, color='red', linestyle='--', label=f'Mean: {mean_time*1000:.2f}ms')
            axes[1,1].set_title('Inference Time Distribution')
            axes[1,1].set_xlabel('Inference Time (ms)')
            axes[1,1].set_ylabel('Frequency')
            axes[1,1].legend()
        
        # Plot 6: System Summary
        axes[1,2].text(0.1, 0.8, 'AmbuRoute Test Summary', fontsize=14, fontweight='bold')
        
        if 'detection_metrics' in self.test_results:
            dm = self.test_results['detection_metrics']
            summary_text = f"""
Precision: {dm['precision']:.3f}
Recall: {dm['recall']:.3f}
F1-Score: {dm['f1_score']:.3f}
Accuracy: {dm['accuracy']:.3f}

Total Images: {dm['total_images']}
True Positives: {dm['true_positives']}
False Positives: {dm['false_positives']}
False Negatives: {dm['false_negatives']}
            """
            axes[1,2].text(0.1, 0.6, summary_text, fontsize=10, verticalalignment='top')
        
        axes[1,2].set_xlim(0, 1)
        axes[1,2].set_ylim(0, 1)
        axes[1,2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Test results plotted successfully!")

# Initialize tester
print("✅ AmbuRoute Tester ready!")


✅ AmbuRoute Tester ready!
